In [1]:
import pandas as pd
import numpy as np
import nltk  
import numpy as np  
import random  
import string
from bs4 import BeautifulSoup
import bs4 as bs  
import urllib.request  
import re  
import os
from zipfile import ZipFile

import catboost
from catboost import CatBoostRegressor
from catboost import Pool, cv
from sklearn.model_selection import train_test_split


from datetime import datetime


pd.set_option('display.max_colwidth', -1)

In [2]:
def zip_files(source_dir):
    for filename in os.listdir(source_dir):
        if filename.endswith('.zip'):
            yield filename
            
def clean_string(liste):
    #liste = Flatten(liste)
    #liste = " ".join(liste)
    liste =  re.sub(r"[\W\_]|\d+", ' ', liste)
    liste = " ".join(liste.split())
    liste = liste.lower()
    
    return liste

def Flatten(ul):
    fl = []
    for i in ul:
        if type(i) is list:
            fl += Flatten(i)
        else:
            fl += [i]
    return fl

def get_main_text(soup):
    text = soup.find(id="ad_description_text").text
    text = clean_string(text)
    cut_string = "html ad container nextsibling"
    return text.split(cut_string,1)[1] 

In [3]:
#Get text
source_dir = r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files'
data_dict ={}
data = {}
i= 0
file_num = len(os.listdir(source_dir))
file_num = 1

file_names = []
all_text = []

for file in zip_files(source_dir):
    print(i, end='\r')
    file_dir = source_dir + "\\" + file
    with ZipFile(file=file_dir) as zip_file:
        try:
            bs = zip_file.read(zip_file.namelist()[0]).decode('utf8')
            soup = BeautifulSoup(bs, "html.parser")
            text = get_main_text(soup)
            all_text.append(text)
            file_names.append(file)
        except:
            pass
        i += 1


In [4]:
text_df = pd.DataFrame(all_text)
text_df["name"] = file_names
text_df.columns = ["text", "name"]

In [5]:
filtered_df = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\filtered_df.csv', usecols=["gesamtmiete", "direct_link"])
text_df["name"]= "https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-" + text_df["name"].str.lower().str.slice(0,-4) + ".html"
merged = pd.merge(filtered_df,text_df, left_on='direct_link', right_on='name')
merged = merged.drop(columns=["direct_link"])
merged['name_length']  = merged['text'].str.len()
merged= merged.query("name_length > 50")
merged['rank'] = pd.qcut(merged['gesamtmiete'], 5, labels=False)

In [11]:
merged["rank"].value_counts()

3    2909
1    2677
0    2553
2    2345
4    2141
Name: rank, dtype: int64

In [6]:
merged.to_csv("housing_text.csv")

In [47]:
len(merged)

11801

In [3]:
source_dir = r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files'
data_dict ={}
data = {}
i= 0
file_num = len(os.listdir(source_dir))
#file_num = 100

file_names = []

for file in zip_files(source_dir):
    print(i, end='\r')
    file_dir = source_dir + "\\" + file
    with ZipFile(file=file_dir) as zip_file:
        try:
            bs = zip_file.read(zip_file.namelist()[0]).decode('utf8')
            soup = BeautifulSoup(bs, "html.parser")
            text = get_main_text(soup)
            for word in text.split():
                val = data.get(word, [0] * file_num)
                #print(i)
                val[i] += 1
                data[word] = val
            file_names.append(file)
            if i == file_num-1:
                break
        except:
            file_names.append(file)
            if i == file_num-1:
                break
            pass
        i += 1


In [4]:
df = pd.DataFrame(data)
df.replace(0, np.nan, inplace=True)

In [5]:
df.memory_usage().sum() / 1e3

6498505.344

In [6]:
df = df.astype(pd.SparseDtype("float", np.nan))

In [7]:
df.memory_usage().sum() / 1e3

27772.124

In [8]:
df = df.dropna(thresh=len(df) - len(df)*0.995, axis=1)

In [9]:
df.memory_usage().sum() / 1e3

24194.108

In [10]:
df["file"] = file_names
filtered_df = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\filtered_df.csv', usecols=["gesamtmiete", "direct_link"])
df["file"]= "https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-" + df["file"].str.lower().str.slice(0,-4) + ".html"

In [11]:
merged = pd.merge(filtered_df,df, left_on='direct_link', right_on='file')